In [277]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
# 데이터 읽어서 DataFrame에 저장
train_df = pd.read_csv("ratings_train.txt", "\t")
test_df = pd.read_csv("ratings_test.txt", "\t")

In [278]:
train_df['document'].nunique(), train_df['label'].nunique()
train_df.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_df = train_df.dropna(how = 'any') # Null 값이 존재하는 행 제거

# 일단 개수를 줄여서 돌려보자.
## train_x 는 series of sentences
## train_y 는 serise of labels

In [279]:
train_size  = 1500  # size 상수
const_tfidf = 500   #구할 tfidf상위 단어의 개수
const_kai   = 500   #구할 카이제곱상위 단어의 개수
test_size   = int(train_size/3)
train_x, train_y = train_df["document"][:train_size], train_df["label"][:train_size]
test_x, test_y = test_df["document"][:test_size], test_df["label"][:test_size]

# 토크나이저
### 뜻이 없는 Josa, Punctuation, Number, Modifier, Eomi 제외

In [280]:
from konlpy.tag import Twitter
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        ret = []
        for word, tag in pos:
            if not(tag == 'Josa' or tag == 'Punctuation' or tag == 'Number' or tag == 'Modifier' or tag == 'Eomi'):
                ret.append(word)
        return ret
my_tokenizer = MyTokenizer(Twitter())

/home/inns/.local/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


# 긍정 문서, 부정 문서 나누기
### 레이블이 이상하게 붙어있는 것들이 있다. 

In [281]:
docNP = [str(train_x[train_y == 1]), str(train_x[train_y == 0])]

In [282]:
print(train_x[56])
print(train_y[56])

최고
0


# 긍/ 부정 문서 tfidf 상위단어 구하기

In [283]:
tfidf = TfidfVectorizer(tokenizer = my_tokenizer, max_features = const_tfidf, max_df = 0.97, min_df = 0.03)
word2vec = tfidf.fit_transform(docNP)
tfidf_dict = tfidf.get_feature_names()

# 긍/ 부정 문서 카이제곱 상위단어 구하기

In [284]:
# def wordCount(data, w):
#     cnt = 0;
#     for s in data:
#         if w in s:
#             cnt += 1
#     return cnt;

# def getKai(data, w, c):
#     A = wordCount(data[c], w)
#     B = wordCount(data[(c+1)%2], w)
#     C = len(data[c]) - A
#     D = len(data[(c+1)%2]) - B
#     # 분모가 0이면 0을 리턴함.
#     if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
#         return 0
#     return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))

In [285]:
# import pickle
# with open('train_moong.pickle', 'rb') as f:
#     doc = pickle.load(f) # 단 한줄씩 읽어옴

In [286]:
# with open('train_label_moong.pickle', 'rb') as f:
#     doc_label = pickle.load(f) # 단 한줄씩 읽어옴

In [287]:
# doc = doc[:train_size]

In [288]:
# words = []
# for s in doc:
#     for w in s:
#         words.append(w)

In [289]:
# words = list(set(words))

In [290]:
# data = [list(train_x[train_y == 0]), list(train_x[train_y == 1])]

In [291]:
# kai_dict = []
# for w in tqdm(words):
#     kai_dict.append(getKai(data, w, 0))

# kaiRate = pd.DataFrame()
# kaiRate['words'] = words
# kaiRate.index = words
# kaiRate['word_kai'] = kai_dict

# kaiRate = kaiRate.sort_values(by=['word_kai'],ascending=False)






  0%|          | 0/101188 [00:00<?, ?it/s]




  1%|          | 1022/101188 [00:00<00:09, 10213.51it/s]




  2%|▏         | 2014/101188 [00:00<00:09, 10122.63it/s]




  3%|▎         | 3000/101188 [00:00<00:09, 10041.17it/s]




  4%|▍         | 4022/101188 [00:00<00:09, 10092.73it/s]




  5%|▍         | 5036/101188 [00:00<00:09, 10105.51it/s]




  6%|▌         | 6059/101188 [00:00<00:09, 10141.49it/s]




  7%|▋         | 7084/101188 [00:00<00:09, 10172.51it/s]




  8%|▊         | 8104/101188 [00:00<00:09, 10180.43it/s]




  9%|▉         | 9143/101188 [00:00<00:08, 10241.84it/s]




 10%|█         | 10165/101188 [00:01<00:08, 10233.83it/s]




 11%|█         | 11191/101188 [00:01<00:08, 10240.33it/s]




 12%|█▏        | 12210/101188 [00:01<00:08, 10223.27it/s]




 13%|█▎        | 13223/101188 [00:01<00:08, 10192.80it/s]




 14%|█▍        | 14247/101188 [00:01<00:08, 10206.26it/s]




 15%|█▌        | 15277/101188 [00:01<00:08, 10234.03it/s]




 16%|█▌        | 16296/1011

In [292]:
kaiRate = kaiRate[:const_kai]

In [293]:
kaiRate = list(kaiRate["words"])

# 단어 개수 카운트

In [294]:
from scipy import sparse
def create_vec_count(x_data, voca):
    vec = np.ndarray((len(x_data), len(voca)))
    for i, s in enumerate(tqdm(x_data)): 
        for j, w in enumerate(voca):
            vec[i][j] = s.count(w)
#     return sparse.csr_matrix(vec)
    return vec

# 원핫

In [295]:
def create_vec_sparse(x_data, voca):
    vec = np.ndarray((len(x_data), len(voca)))
    for i, s in enumerate(tqdm(x_data)): 
        for j, w in enumerate(voca):
            if str(w) in str(s):
                vec[i][j] = 1
            else:
                vec[i][j] = 0
#     return sparse.csr_matrix(vec)
    return vec

In [296]:
train_bone_sparse = create_vec_sparse(train_x, words)
train_tfidf_sparse = create_vec_sparse(train_x, tfidf_dict)
# train_kai_sparse = create_vec_sparse(train_x, kaiRate)
test_bone_sparse = create_vec_sparse(test_x, words)
test_tfidf_sparse = create_vec_sparse(test_x, tfidf_dict)
# test_kai_sparse = create_vec_sparse(test_x, kaiRate)






  0%|          | 0/1500 [00:00<?, ?it/s]




  0%|          | 2/1500 [00:00<01:33, 16.07it/s]




  0%|          | 4/1500 [00:00<01:29, 16.69it/s]




  0%|          | 6/1500 [00:00<01:28, 16.92it/s]




  1%|          | 8/1500 [00:00<01:26, 17.17it/s]




  1%|          | 10/1500 [00:00<01:25, 17.48it/s]




  1%|          | 12/1500 [00:00<01:24, 17.69it/s]




  1%|          | 14/1500 [00:00<01:23, 17.72it/s]




  1%|          | 16/1500 [00:00<01:23, 17.85it/s]




  1%|          | 18/1500 [00:01<01:23, 17.81it/s]




  1%|▏         | 20/1500 [00:01<01:22, 17.85it/s]




  1%|▏         | 22/1500 [00:01<01:22, 17.96it/s]




  2%|▏         | 24/1500 [00:01<01:24, 17.57it/s]




  2%|▏         | 26/1500 [00:01<01:22, 17.78it/s]




  2%|▏         | 28/1500 [00:01<01:21, 18.07it/s]




  2%|▏         | 30/1500 [00:01<01:20, 18.24it/s]




  2%|▏         | 32/1500 [00:01<01:20, 18.23it/s]




  2%|▏         | 34/1500 [00:01<01:20, 18.23it/s]




  2%|▏         | 36/1500 [00:02<01:2

 16%|█▌        | 234/1500 [00:13<01:14, 17.04it/s]




 16%|█▌        | 236/1500 [00:13<01:12, 17.38it/s]




 16%|█▌        | 238/1500 [00:13<01:12, 17.51it/s]




 16%|█▌        | 240/1500 [00:13<01:10, 17.76it/s]




 16%|█▌        | 242/1500 [00:13<01:10, 17.93it/s]




 16%|█▋        | 244/1500 [00:13<01:09, 17.96it/s]




 16%|█▋        | 246/1500 [00:13<01:09, 18.04it/s]




 17%|█▋        | 248/1500 [00:13<01:08, 18.15it/s]




 17%|█▋        | 250/1500 [00:13<01:08, 18.22it/s]




 17%|█▋        | 252/1500 [00:14<01:08, 18.12it/s]




 17%|█▋        | 254/1500 [00:14<01:07, 18.34it/s]




 17%|█▋        | 256/1500 [00:14<01:08, 18.26it/s]




 17%|█▋        | 258/1500 [00:14<01:07, 18.29it/s]




 17%|█▋        | 260/1500 [00:14<01:08, 18.21it/s]




 17%|█▋        | 262/1500 [00:14<01:08, 18.02it/s]




 18%|█▊        | 264/1500 [00:14<01:08, 18.02it/s]




 18%|█▊        | 266/1500 [00:14<01:07, 18.21it/s]




 18%|█▊        | 268/1500 [00:14<01:08, 18.04it/s]




 18%|█▊   

 31%|███       | 468/1500 [00:26<00:57, 17.94it/s]




 31%|███▏      | 470/1500 [00:26<00:57, 17.93it/s]




 31%|███▏      | 472/1500 [00:26<00:56, 18.19it/s]




 32%|███▏      | 474/1500 [00:26<00:57, 17.93it/s]




 32%|███▏      | 476/1500 [00:26<00:57, 17.92it/s]




 32%|███▏      | 478/1500 [00:26<00:56, 17.95it/s]




 32%|███▏      | 480/1500 [00:26<00:56, 18.11it/s]




 32%|███▏      | 482/1500 [00:26<00:56, 18.11it/s]




 32%|███▏      | 484/1500 [00:26<00:55, 18.15it/s]




 32%|███▏      | 486/1500 [00:27<00:55, 18.23it/s]




 33%|███▎      | 488/1500 [00:27<00:55, 18.30it/s]




 33%|███▎      | 490/1500 [00:27<00:55, 18.29it/s]




 33%|███▎      | 492/1500 [00:27<00:56, 17.90it/s]




 33%|███▎      | 494/1500 [00:27<00:55, 18.12it/s]




 33%|███▎      | 496/1500 [00:27<00:55, 18.04it/s]




 33%|███▎      | 498/1500 [00:27<00:56, 17.89it/s]




 33%|███▎      | 500/1500 [00:27<00:55, 17.94it/s]




 33%|███▎      | 502/1500 [00:27<00:54, 18.15it/s]




 34%|███▎ 

 47%|████▋     | 702/1500 [00:39<01:08, 11.60it/s]




 47%|████▋     | 704/1500 [00:39<01:11, 11.20it/s]




 47%|████▋     | 706/1500 [00:39<01:05, 12.10it/s]




 47%|████▋     | 708/1500 [00:39<00:59, 13.29it/s]




 47%|████▋     | 710/1500 [00:39<00:54, 14.41it/s]




 47%|████▋     | 712/1500 [00:40<00:52, 15.00it/s]




 48%|████▊     | 714/1500 [00:40<00:50, 15.53it/s]




 48%|████▊     | 716/1500 [00:40<00:48, 16.30it/s]




 48%|████▊     | 718/1500 [00:40<00:47, 16.37it/s]




 48%|████▊     | 720/1500 [00:40<00:46, 16.72it/s]




 48%|████▊     | 722/1500 [00:40<00:44, 17.30it/s]




 48%|████▊     | 724/1500 [00:40<00:44, 17.44it/s]




 48%|████▊     | 726/1500 [00:40<00:43, 17.60it/s]




 49%|████▊     | 728/1500 [00:40<00:43, 17.67it/s]




 49%|████▊     | 730/1500 [00:41<00:44, 17.47it/s]




 49%|████▉     | 732/1500 [00:41<00:43, 17.61it/s]




 49%|████▉     | 734/1500 [00:41<00:42, 17.94it/s]




 49%|████▉     | 736/1500 [00:41<00:42, 18.11it/s]




 49%|████▉

 62%|██████▏   | 936/1500 [00:52<00:30, 18.22it/s]




 63%|██████▎   | 938/1500 [00:52<00:31, 17.93it/s]




 63%|██████▎   | 940/1500 [00:52<00:31, 17.98it/s]




 63%|██████▎   | 942/1500 [00:52<00:31, 17.74it/s]




 63%|██████▎   | 944/1500 [00:53<00:31, 17.81it/s]




 63%|██████▎   | 946/1500 [00:53<00:30, 18.00it/s]




 63%|██████▎   | 948/1500 [00:53<00:30, 18.11it/s]




 63%|██████▎   | 950/1500 [00:53<00:30, 18.33it/s]




 63%|██████▎   | 952/1500 [00:53<00:30, 17.78it/s]




 64%|██████▎   | 954/1500 [00:53<00:30, 17.91it/s]




 64%|██████▎   | 956/1500 [00:53<00:30, 17.69it/s]




 64%|██████▍   | 958/1500 [00:53<00:30, 17.70it/s]




 64%|██████▍   | 960/1500 [00:53<00:30, 17.97it/s]




 64%|██████▍   | 962/1500 [00:54<00:29, 18.13it/s]




 64%|██████▍   | 964/1500 [00:54<00:29, 17.96it/s]




 64%|██████▍   | 966/1500 [00:54<00:29, 18.00it/s]




 65%|██████▍   | 968/1500 [00:54<00:29, 18.18it/s]




 65%|██████▍   | 970/1500 [00:54<00:29, 17.70it/s]




 65%|█████

 78%|███████▊  | 1166/1500 [01:06<00:28, 11.53it/s]




 78%|███████▊  | 1168/1500 [01:06<00:28, 11.52it/s]




 78%|███████▊  | 1170/1500 [01:06<00:26, 12.50it/s]




 78%|███████▊  | 1172/1500 [01:06<00:25, 12.99it/s]




 78%|███████▊  | 1174/1500 [01:06<00:24, 13.45it/s]




 78%|███████▊  | 1176/1500 [01:06<00:22, 14.40it/s]




 79%|███████▊  | 1178/1500 [01:06<00:21, 15.01it/s]




 79%|███████▊  | 1180/1500 [01:07<00:20, 15.84it/s]




 79%|███████▉  | 1182/1500 [01:07<00:19, 16.40it/s]




 79%|███████▉  | 1184/1500 [01:07<00:18, 16.76it/s]




 79%|███████▉  | 1186/1500 [01:07<00:18, 17.19it/s]




 79%|███████▉  | 1188/1500 [01:07<00:17, 17.49it/s]




 79%|███████▉  | 1190/1500 [01:07<00:17, 17.79it/s]




 79%|███████▉  | 1192/1500 [01:07<00:17, 17.84it/s]




 80%|███████▉  | 1194/1500 [01:07<00:17, 17.78it/s]




 80%|███████▉  | 1196/1500 [01:07<00:16, 18.03it/s]




 80%|███████▉  | 1198/1500 [01:08<00:16, 17.83it/s]




 80%|████████  | 1200/1500 [01:08<00:16, 17.71it

 93%|█████████▎| 1396/1500 [01:19<00:05, 17.90it/s]




 93%|█████████▎| 1398/1500 [01:19<00:05, 18.02it/s]




 93%|█████████▎| 1400/1500 [01:19<00:05, 17.72it/s]




 93%|█████████▎| 1402/1500 [01:19<00:05, 17.72it/s]




 94%|█████████▎| 1404/1500 [01:19<00:05, 17.97it/s]




 94%|█████████▎| 1406/1500 [01:19<00:05, 18.08it/s]




 94%|█████████▍| 1408/1500 [01:19<00:05, 18.12it/s]




 94%|█████████▍| 1410/1500 [01:19<00:04, 18.29it/s]




 94%|█████████▍| 1412/1500 [01:19<00:04, 18.18it/s]




 94%|█████████▍| 1414/1500 [01:20<00:04, 18.23it/s]




 94%|█████████▍| 1416/1500 [01:20<00:04, 18.25it/s]




 95%|█████████▍| 1418/1500 [01:20<00:04, 18.13it/s]




 95%|█████████▍| 1420/1500 [01:20<00:04, 18.34it/s]




 95%|█████████▍| 1422/1500 [01:20<00:04, 18.46it/s]




 95%|█████████▍| 1424/1500 [01:20<00:04, 18.30it/s]




 95%|█████████▌| 1426/1500 [01:20<00:04, 17.97it/s]




 95%|█████████▌| 1428/1500 [01:20<00:04, 17.81it/s]




 95%|█████████▌| 1430/1500 [01:20<00:03, 17.58it

 24%|██▍       | 120/500 [00:06<00:20, 18.50it/s]




 24%|██▍       | 122/500 [00:06<00:20, 18.58it/s]




 25%|██▍       | 124/500 [00:06<00:20, 18.53it/s]




 25%|██▌       | 126/500 [00:06<00:20, 18.35it/s]




 26%|██▌       | 128/500 [00:07<00:20, 18.14it/s]




 26%|██▌       | 130/500 [00:07<00:20, 18.13it/s]




 26%|██▋       | 132/500 [00:07<00:20, 18.27it/s]




 27%|██▋       | 134/500 [00:07<00:20, 17.93it/s]




 27%|██▋       | 136/500 [00:07<00:20, 17.48it/s]




 28%|██▊       | 138/500 [00:07<00:20, 17.81it/s]




 28%|██▊       | 140/500 [00:07<00:20, 17.63it/s]




 28%|██▊       | 142/500 [00:07<00:19, 17.95it/s]




 29%|██▉       | 144/500 [00:07<00:19, 18.24it/s]




 29%|██▉       | 146/500 [00:08<00:19, 18.14it/s]




 30%|██▉       | 148/500 [00:08<00:19, 18.17it/s]




 30%|███       | 150/500 [00:08<00:19, 18.11it/s]




 30%|███       | 152/500 [00:08<00:19, 17.75it/s]




 31%|███       | 154/500 [00:08<00:19, 17.96it/s]




 31%|███       | 156/500 [00

 71%|███████   | 356/500 [00:19<00:07, 18.26it/s]




 72%|███████▏  | 358/500 [00:19<00:07, 18.29it/s]




 72%|███████▏  | 360/500 [00:19<00:07, 18.43it/s]




 72%|███████▏  | 362/500 [00:19<00:07, 18.37it/s]




 73%|███████▎  | 364/500 [00:20<00:07, 18.41it/s]




 73%|███████▎  | 366/500 [00:20<00:07, 18.24it/s]




 74%|███████▎  | 368/500 [00:20<00:07, 18.12it/s]




 74%|███████▍  | 370/500 [00:20<00:07, 17.86it/s]




 74%|███████▍  | 372/500 [00:20<00:07, 17.91it/s]




 75%|███████▍  | 374/500 [00:20<00:07, 17.99it/s]




 75%|███████▌  | 376/500 [00:20<00:06, 18.14it/s]




 76%|███████▌  | 378/500 [00:20<00:06, 18.18it/s]




 76%|███████▌  | 380/500 [00:20<00:06, 17.89it/s]




 76%|███████▋  | 382/500 [00:21<00:06, 18.07it/s]




 77%|███████▋  | 384/500 [00:21<00:06, 18.25it/s]




 77%|███████▋  | 386/500 [00:21<00:06, 18.40it/s]




 78%|███████▊  | 388/500 [00:21<00:06, 18.28it/s]




 78%|███████▊  | 390/500 [00:21<00:05, 18.69it/s]




 78%|███████▊  | 392/500 [00

In [297]:
# train_bone_cnt = create_vec_count(train_x, words)
# train_tfidf_cnt = create_vec_count(train_x, tfidf_dict)
# train_kai_cnt = create_vec_count(train_x, kaiRate)
# test_bone_cnt = create_vec_count(test_x, words)
# test_tfidf_cnt = create_vec_count(test_x, tfidf_dict)
# test_kai_cnt = create_vec_count(test_x, kaiRate)

In [299]:
# 벡터를 합친다.
def merge_sparse(a, b):
    return sparse.csr_matrix(np.hstack((a.toarray(), b.toarray())))
def merge_np(a, b):
    return np.concatenate((a, b), axis = 1)
train_with_tfidf_sprs = merge_np(train_bone_sparse, train_tfidf_sparse)
# train_with_tfidf_sprs = merge_sparse(train_bone_sparse, train_tfidf_sparse)
# train_with_tfidf_cnt = merge_sparse(train_bone_cnt, train_tfidf_cnt)
test_with_tfidf_sprs = merge_np(test_bone_sparse, test_tfidf_sparse)
# test_with_tfidf_sprs = merge_sparse(test_bone_sparse, test_tfidf_sparse)
# test_with_tfidf_cnt = merge_sparse(test_bone_cnt, test_tfidf_cnt)

# train_with_kai_sprs = merge_sparse(train_bone_sparse, train_kai_sparse)
# train_with_kai_cnt = merge_sparse(train_bone_cnt, train_kai_cnt)

# test_with_kai_sprs = merge_sparse(test_bone_sparse, test_kai_sparse)
# test_with_kai_cnt = merge_sparse(test_bone_cnt, test_kai_cnt)

# train_with_tfidf_kai_sprs   = merge_sparse(train_with_tfidf_sprs, train_kai_sparse)
# train_with_tfidf_kai_cnt    = merge_sparse(train_with_tfidf_cnt, train_kai_cnt)

# test_with_tfidf_kai_sprs    = merge_sparse(test_with_tfidf_sprs, test_kai_sparse) 
# test_with_tfidf_kai_cnt     = merge_sparse(test_with_tfidf_cnt, test_kai_cnt)

In [300]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
classifier_tfidf_sprs = MultinomialNB()
classifier_tfidf_sprs.fit(train_with_tfidf_sprs, train_y)

pred_tfidf_sprs = classifier_tfidf_sprs.predict(test_with_tfidf_sprs).tolist()
print('Accuracy: %.10f' % accuracy_score(test_y, pred_tfidf_sprs))

Accuracy: 0.7680000000


In [301]:
# classifier_tfidf_cnt = MultinomialNB()
# classifier_tfidf_cnt.fit(train_with_tfidf_cnt, train_y)

# pred_tfidf_cnt = classifier_tfidf_sprs.predict(test_with_tfidf_cnt).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_tfidf_cnt))

Accuracy: 0.6140000000


In [302]:
# classifier_tfidf_cnt = MultinomialNB()
# classifier_tfidf_cnt.fit(train_with_tfidf_cnt, train_y)

# pred_tfidf_cnt = classifier_tfidf_sprs.predict(test_with_tfidf_cnt).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_tfidf_cnt))

In [303]:
# classifier_kai_sprs = MultinomialNB()
# classifier_kai_sprs.fit(train_with_kai_sprs, train_y)

# pred_kai_sprs = classifier_kai_sprs.predict(test_with_kai_sprs).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_kai_sprs))

In [304]:
# classifier_kai_cnt = MultinomialNB()
# classifier_kai_cnt.fit(train_with_kai_cnt, train_y)

# pred_kai_cnt = classifier_kai_cnt.predict(test_with_kai_cnt).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_kai_cnt))

In [305]:
# classifier_tfidf_kai_sprs = MultinomialNB()
# classifier_tfidf_kai_sprs.fit(train_with_tfidf_kai_sprs, train_y)

# pred_tfidf_kai_sprs = classifier_tfidf_kai_sprs.predict(test_with_tfidf_kai_sprs).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_tfidf_kai_sprs))

In [306]:
# classifier_tfidf_kai_cnt = MultinomialNB()
# classifier_tfidf_kai_cnt.fit(train_with_tfidf_kai_cnt, train_y)

# pred_tfidf_kai_cnt = classifier_tfidf_kai_cnt.predict(test_with_tfidf_kai_cnt).tolist()
# print('Accuracy: %.10f' % accuracy_score(test_y, pred_tfidf_kai_cnt))

In [307]:
print("tfidf_sprs     : %.10f" % accuracy_score(test_y, pred_tfidf_sprs))
# print("kai_sprs       : %.10f" % accuracy_score(test_y, pred_kai_sprs))
# print("tfidf+kai_sprs : %.10f" % accuracy_score(test_y, pred_tfidf_kai_sprs))
# print("tfidf_cnt      : %.10f" % accuracy_score(test_y, pred_tfidf_cnt))
# print("kai_cnt        : %.10f" % accuracy_score(test_y, pred_kai_cnt))
# print("tfidf+kai_cnt  : %.10f" % accuracy_score(test_y, pred_tfidf_kai_cnt))

tfidf_sprs     : 0.7680000000
